# `astroquery`: NASA Exoplanet Archive

Say you want to make a mass-radius diagram of known exoplanets, with points colored by the host star's effective temperature. If you were to do this by hand, you might: 

* download a table of the known exoplanets and their properties
* parse the table
* add units to each quantity
* plot the results

Fortunately, a lot of this functionality is already implemented in a astropy affiliated package called [`astroquery`](https://astroquery.readthedocs.io/en/latest/)! Here's a few lines of code that do all of the above tasks for you: 

In [ ]:
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive

# Downloads the NASA Exoplanet Archive confirmed planets table, 
# and returns an astropy QTable object

archive = NasaExoplanetArchive.get_confirmed_planets_table(all_columns=True)

Let's inspect the object that gets returned:

In [ ]:
archive

The astropy `QTable` object has columns containing parameters relevant to each planet, and units for each column where applicable. You can access a column of the astropy table like so: 

In [ ]:
archive['pl_orbper']  # Print the orbital periods of the known exoplanets

Let's now make that mass-radius diagram that we set out to make: 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt 
import numpy as np

cax = plt.scatter(np.log10(archive['pl_bmassj'].value), archive['pl_radj'], c=archive['st_teff'], 
                  vmin=3000, vmax=8000)
cbar = plt.colorbar(cax, label='T$_\mathrm{eff}$ [K]', extend='both')
ax = plt.gca()
ax.set(xlim=[-3, 2], ylim=[0.1, 2.5], 
       xlabel='log$_{10}$ Mass [M$_J$]', ylabel='Radius [R$_J$]')

Or you could make your own version of the famous planet discovery diagram: 

In [ ]:
transits = archive['pl_discmethod'] == 'Transit'
rvs = archive['pl_discmethod'] == 'Radial Velocity'

plt.scatter(np.log10(archive['pl_orbper'][transits].value), 
            archive['pl_radj'][transits], label='Transit')
plt.scatter(np.log10(archive['pl_orbper'][rvs].value),
            archive['pl_radj'][rvs], label='RV')
plt.gca().set(xlabel='log$_{10}$ Period [d]', ylabel='Radius', 
              xlim=[-1, 3.5], ylim=[0.05, 2.1])
plt.legend(loc='upper right')

As you can see, by default this astropy `QTable` returns columns with units, so you can do whatever arithmetic you like on the numbers it returns. For example, let's compute the exoplanet equilibrium temperatures, which is given by 

$$ T_\mathrm{eq} = T_\mathrm{eff} \sqrt{\frac{R_\star}{2a}} (1 - A_B)^{1/4} $$

assuming $A_B = 0$: 

In [ ]:
teff = archive['st_teff']  # stellar effective temperature
rstar = archive['st_rad']  # stellar radius
a = archive['pl_orbsmax']  # orbital semimajor axis

teq = (teff * np.sqrt(rstar / (2 * a))).decompose()  # Calling decompose returns the "natural" units of Kelvin

plt.hist(teq.value[np.isfinite(teq)], log=True, bins=50, range=[10, 5000])
plt.gca().set(xlabel='T$_\mathrm{eq}$ [K]', ylabel='Frequency')